In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
from sapy.components import Clock, ProgramCounter, MemoryAddressRegister, \
    RandomAccessMemory, SwitchBoard, RegisterA, RegisterB, \
    RegisterOutput, ArithmeticUnit, RegisterInstruction, Computer, opcode_map
from sapy.assembler import assemble

# Hex and Binary Literals in Python

In [3]:
0xF

15

In [4]:
0xFF

255

In [5]:
print(15_000) # underscore as separators are allowed
print(0b1111_1111)

15000
255


You can also format integers to HEX using X

In [6]:
f"0x{232:02X}"

'0xE8'

# Assembly Language

In [7]:
assemble("LDA #$07")

0x00  20 07   # LDA #$07  


[32, 7]

# Clocking Signals

In [8]:
clk = Clock()
class DummyComponent():
    def data(self, con=tuple()):
        print("data accessed")
        return 42
    def clock(self, data, con=tuple()):
        print("clock stepped")

dc = DummyComponent()
clk.add_component(dc)

In [9]:
print("step 1: ---------")
clk.step()
print("step 2: ---------")
clk.step()

step 1: ---------
data accessed
------------------------------------------
PCADDRESS: $2A
T0: Data: $2A, Control Word: ('ep', 'lm', 'cp')
clock stepped
step 2: ---------
data accessed
T1: Data: $2A, Control Word: ('er', 'li')
clock stepped


# Program Counter

To enable a component to take an action on a clock, you pass in the correct control bit as a keyword argument. To enable the PC to output to the data but, use the "ep" control bit.

In [10]:
pc = ProgramCounter()

In [11]:
pc.clock()
pc.data()

In [12]:
pc.clock()
pc.data(con=['ep'])

0

In [13]:
pc.clock()
pc.data(con=['ep'])

0

To increment the program counter, the control bit is "cp".

In [14]:
pc.clock(con=['cp'])
pc.data(con=['ep'])

1

In [15]:
pc.clock(con=['cp'])
pc.data(con=['ep'])

2

In [16]:
pc.clock(con=['cp'])
pc.data(con=['ep'])

3

For jump instructions you set the program counter directly. This is done with the "lp" control bit.

In [17]:
pc.clock(data=0xD, con=['lp'])
print(f"0x{pc.data(con=['ep']):02X}")
print(f"{pc.data(con=['ep'])}")

0x0D
13


# Accumulator (Register A)

In [18]:
reg_a = RegisterA()
print(f"The register value on reset is: 0x{reg_a.data(con=['ea']):0X}")

reg_a.clock(data=0xAB, con=['la'])
reg_a.data(con=['ea'])
print(f"The register has latched a value: 0x{reg_a.data(con=['ea']):0X}")


The register value on reset is: 0x0
The register has latched a value: 0xAB


# ALU
## Arithmatic Logic Unit

Some components have extra attributes that allow it to communicate directly to other components outside of the bus. 

The ALU is not clocked, it doesn't even have state. The data output from the ALU is alway the instantaneous sum (or difference) of Reg A + Reg B

In [19]:
reg_a = RegisterA()
reg_a.clock(data=4, con=['la'])

reg_b = RegisterB()
reg_b.clock(data=3, con=['lb'])

alu = ArithmeticUnit(reg_a, reg_b)

alu.data(con=['eu'])

print(f"\t0x{reg_a.value:02X}")
print(f"+\t0x{reg_b.value:02X}")
print('-' * 12)
print(f"\t0x{alu.data(con=['eu']):02X}")

	0x04
+	0x03
------------
	0x07


In [20]:
# change the value of the register feeding the alu
reg_b.clock(data=1, con=['lb'])
alu.data(con=['eu'])

print(f"\t0x{reg_a.value:02X}")
print(f"+\t0x{reg_b.value:02X}")
print('-' * 12)
print(f"\t0x{alu.data(con=['eu']):02X}")

	0x04
+	0x01
------------
	0x05


Subtraction

In [21]:

print(f"\t0x{reg_a.value:02X}")
print(f"-\t0x{reg_b.value:02X}")
print('-' * 12)
print(f"\t0x{alu.data(con=['eu', 'su']):02X}")

	0x04
-	0x01
------------
	0x03


# Memory

Ram is another example of a system that has communication outside the bus, it always access the RAM value stored at the address latched in the Memory Address Register.

In [22]:
mar = MemoryAddressRegister()
ram = RandomAccessMemory(mar)

# store address for ram in register
mar.clock(data=0x0F, con=['lm'])
print(f"Memory Address: 0x{mar.value:02X}")


# clock data into ram at the address set above
ram.clock(data=0xAB, con=['lr'])
assert ram.data() == None
print(f"Memory at address: 0x{mar.value:02X} is 0x{ram.data(con=['er']):02X}") # should be 0xAB
print(f"Memory at address: {mar.value} is {ram.data(con=['er'])}") # just to demystify hex..

Memory Address: 0x0F
Memory at address: 0x0F is 0xAB
Memory at address: 15 is 171


# Controller: Sequencer + Decoder + Clock

In [23]:
reg_i = RegisterInstruction()
instruction = 0xFF # both opcode and argument, 8bits
reg_i.clock(data=instruction, con=['li'])


In [24]:
print(f"The opcode is {reg_i.value:X}")

The opcode is FF


# Example Sequencing: LDA

In [25]:
from sapy.components import Computer

cpu = Computer()
rom = [
    0x00, 0x09, # 0x0 LDA 09H
    ]
cpu.switches.load_program(rom)

for _ in range(7):
    cpu.step(debug=True) 

------------------------------------------
PCADDRESS: $00
T0: Data: $00, Control Word: ('ep', 'lm', 'cp')
T1: Data: $00, Control Word: ('er', 'li')
OPCODE: $00, MNE: LDA
T2: Data: $01, Control Word: ('ep', 'lm', 'cp')
T3: Data: $09, Control Word: ('er', 'lm')
T4: Data: $00, Control Word: ('er', 'la')
------------------------------------------
PCADDRESS: $02
T0: Data: $02, Control Word: ('ep', 'lm', 'cp')
T1: Data: $00, Control Word: ('er', 'li')
OPCODE: $00, MNE: LDA


In [26]:
cpu.step(debug=True)

T2: Data: $03, Control Word: ('ep', 'lm', 'cp')


# Output (Register O)

In [27]:
reg_o = RegisterOutput()
reg_o.clock(data=0x90, con=['lo'])

Output Display: 90


In [28]:
# monkey patch output function for fancy display
def display_number(x):
    print(f"***### {x} ###***")
reg_o.output_function = display_number

In [29]:
reg_o.clock(data=0x90, con=['lo'])

***### 144 ###***


# Full Computer with ROM

In [30]:
cpu = Computer()
rom = [
    0x00, 0x07, # 0x00 LDA 07H
    0x01, 0x08, # 0x02 ADD 08H
    0xF6,       # 0x04 OTA
    0x34, 0x02, # 0x05 JMP 02H
    0x00,       # 0x07 A1H
    0x03,       # 0x08 22H
    ]
cpu.switches.load_program(rom)

In [31]:
for _ in range(5):
    cpu.step(instructionwise=True)

------------------------------------------
PCADDRESS: $00
T0: Data: $00, Control Word: ('ep', 'lm', 'cp')
T1: Data: $00, Control Word: ('er', 'li')
OPCODE: $00, MNE: LDA
T2: Data: $01, Control Word: ('ep', 'lm', 'cp')
T3: Data: $07, Control Word: ('er', 'lm')
T4: Data: $00, Control Word: ('er', 'la')
------------------------------------------
PCADDRESS: $02
T0: Data: $02, Control Word: ('ep', 'lm', 'cp')
T1: Data: $01, Control Word: ('er', 'li')
OPCODE: $01, MNE: ADD
T2: Data: $03, Control Word: ('ep', 'lm', 'cp')
T3: Data: $08, Control Word: ('er', 'lm')
T4: Data: $03, Control Word: ('er', 'lb')
T5: Data: $03, Control Word: ('eu', 'la')
------------------------------------------
PCADDRESS: $04
T0: Data: $04, Control Word: ('ep', 'lm', 'cp')
T1: Data: $F6, Control Word: ('er', 'li')
OPCODE: $F6, MNE: OTA
T2: Data: $03, Control Word: ('ea', 'lo')
Output Display: 3
------------------------------------------
PCADDRESS: $05
T0: Data: $05, Control Word: ('ep', 'lm', 'cp')
T1: Data: $34, Con

In [32]:
for _ in range(3):
    cpu.step(instructionwise=True, debug=True)

------------------------------------------
PCADDRESS: $04
T0: Data: $04, Control Word: ('ep', 'lm', 'cp')
T1: Data: $F6, Control Word: ('er', 'li')
OPCODE: $F6, MNE: OTA
T2: Data: $06, Control Word: ('ea', 'lo')
Output Display: 6
------------------------------------------
PCADDRESS: $05
T0: Data: $05, Control Word: ('ep', 'lm', 'cp')
T1: Data: $34, Control Word: ('er', 'li')
OPCODE: $34, MNE: JMP
T2: Data: $06, Control Word: ('ep', 'lm', 'cp')
T3: Data: $02, Control Word: ('er', 'lp')
------------------------------------------
PCADDRESS: $02
T0: Data: $02, Control Word: ('ep', 'lm', 'cp')
T1: Data: $01, Control Word: ('er', 'li')
OPCODE: $01, MNE: ADD
T2: Data: $03, Control Word: ('ep', 'lm', 'cp')
T3: Data: $08, Control Word: ('er', 'lm')
T4: Data: $03, Control Word: ('er', 'lb')
T5: Data: $09, Control Word: ('eu', 'la')


In [33]:
import ipywidgets as widgets
from IPython.display import display
gui_output_display = widgets.FloatText()
display(gui_output_display)

# monkey patch output function
def display_number(x):
    gui_output_display.value = x

cpu.reg_o.output_function = display_number

FloatText(value=0.0)

In [34]:
for _ in range(3):
    time.sleep(.2)
    cpu.step(instructionwise=True, debug=False)

In [35]:
for _ in range(27):
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

In [36]:
for _ in range(133):
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

# DMA

In [62]:
import numpy as np
import ipywidgets as widgets
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [38]:
def setup_bokeh_gui(cpu):
    # Zeroed-out memory map for initializing the display
    initial_bitmap = np.zeros((0x10, 0x10))
    
    # store the actual address for use in hovertool
    address_bitmap = np.ndarray((0x10, 0x10), dtype='int')
    for i in range(initial_bitmap.shape[0]):
        for j in range(initial_bitmap.shape[1]):
            address_bitmap[i, j] = i * 0x10 + j
            
    hover = HoverTool(
        tooltips=[("Address", "@address{$%02X}"),
                  ("value", "@image{$%02X}"),
                 ],
        formatters={
            'address' : 'printf',
            'image' : 'printf',
            },
    )

    p = figure(
        title="Bitmap of Memory",
        plot_height=512,
        plot_width=512,
        tools="",
        background_fill_color='#efefef',
        )
    

    p.add_tools(hover)
    p.toolbar_location = None

    i = p.image(image=[initial_bitmap],
                x=-0.5, y=-0.5, dw=0xF+1, dh=0xF+1, 
                palette="Spectral11",
               )
    i.data_source.data['address'] = [address_bitmap]
    show(p, notebook_handle=True)
    
    
    def plt_memory(bitmap):
        """Refresh the bokeh plot with new bitmap"""
        i.data_source.data['image'] = [bitmap]
        push_notebook()
        
    cpu.dma.connect_dma_handler(plt_memory)

In [39]:
cpu = Computer()

In [40]:
countup = """
    LDA #$99
    ADD #$0E
    DMA
    STA  $4A
    JMP  $02
    """

countup_bytes = assemble(countup)

0x00  20 99   # LDA #$99  
0x02  21 0E   # ADD #$0E  
0x04  FD      # DMA       
0x05  35 4A   # STA  $4A  
0x07  34 02   # JMP  $02  


In [41]:
moveacross = """
    LDA #$10
    loop:
    ADD #$03
    DMA
    STA  colorbuffer
    STA (colorbuffer)
    JMP  loop
    colorbuffer:
    """
moveacross_bytes = assemble(moveacross)

0x00  20 10   # LDA #$10  
0x02  21 03   # ADD #$03  :loop
0x04  FD      # DMA       
0x05  35 0B   # STA  $0B  
0x07  45 0B   # STA ($0B) 
0x09  34 02   # JMP  $02  


In [42]:
cpu.reset()
cpu.switches.load_program(countup_bytes)
setup_bokeh_gui(cpu)

for i in range(400):
    time.sleep(.01)
    if i % 50 == 0:
        print(i)
    cpu.step(instructionwise=True, debug=False)

0
50
100
150
200
250
300
350


In [43]:
cpu = Computer()
cpu.reset()
cpu.switches.load_program(moveacross_bytes)
setup_bokeh_gui(cpu)

for i in range(400):
    #time.sleep(.01)
    cpu.step(instructionwise=True, debug=False)

# Write your own program

In [61]:
program = """
    LDA #locationbuffer
    ADD #$01
    STA locationbuffer
    BAI
    STA colorbuffer
loop:
    DMA
    LDA  colorbuffer
    ADD #$01
    STA  colorbuffer
    
    LDA locationbuffer
    ADD #$02
    STA locationbuffer
    
    LDA colorbuffer
    STA (locationbuffer)
    JMP  loop
colorbuffer:
    BYTE #33 44 55 66
locationbuffer:
    NOP
"""

program_bytes = assemble(program)

cpu = Computer()
cpu.reset()
cpu.switches.load_program(program_bytes)
setup_bokeh_gui(cpu)

for i in range(999):
    time.sleep(.01)
    cpu.step(instructionwise=True, debug=False)

0x00  20 20   # LDA #$20  
0x02  21 01   # ADD #$01  
0x04  35 20   # STA $20   
0x06  F7      # BAI       
0x07  35 1C   # STA $1C   
0x09  FD      # DMA       :loop
0x0A  00 1C   # LDA  $1C  
0x0C  21 01   # ADD #$01  
0x0E  35 1C   # STA  $1C  
0x10  00 20   # LDA $20   
0x12  21 02   # ADD #$02  
0x14  35 20   # STA $20   
0x16  00 1C   # LDA $1C   
0x18  45 20   # STA ($20) 
0x1A  34 09   # JMP  $09  
0x1C  33 44 55 66# BYTE #33 44 55 66:colorbuffer
0x20  FE      # NOP       :locationbuffer


Enter a Hexadecimal number 00 <= x <= FF:
33


In [110]:
import IPython
ipython = IPython.get_ipython()

def setup_ipywidget_input_reg(cpu):
    slider = widgets.IntSlider(value=0x90, min=0x00, max=0xFF, step=1)
    display(slider)
    def update_input_reg():
        ipython.kernel.do_one_iteration()
        return slider.value
    cpu.reg_c.input_function = update_input_reg

In [116]:
program = """
    ;Start the drawing after the last buffer
    LDA #locationbuffer
    ADD #$01
    STA locationbuffer
loop:
    DMA
    
    LDA locationbuffer
    ADD #$02
    STA locationbuffer
    
    BAI
    STA (locationbuffer)
    JMP  loop
colorbuffer:
    BYTE #33 44 55 66
locationbuffer:
    NOP
"""

program_bytes = assemble(program)

cpu = Computer()
cpu.reset()
cpu.switches.load_program(program_bytes)
setup_bokeh_gui(cpu)
setup_ipywidget_input_reg(cpu)


Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
0x00  20 16   # LDA #$16  
0x02  21 01   # ADD #$01  
0x04  35 16   # STA $16   
0x06  FD      # DMA       :loop
0x07  00 16   # LDA $16   
0x09  21 02   # ADD #$02  
0x0B  35 16   # STA $16   
0x0D  F7      # BAI       
0x0E  45 16   # STA ($16) 
0x10  34 06   # JMP  $06  
0x12  33 44 55 66# BYTE #33 44 55 66:colorbuffer
0x16  FE      # NOP       :locationbuffer
ERROR! Session/line number was not unique in database. History logging moved to new session 134


IntSlider(value=144, max=255)

In [117]:
for i in range(819):
    time.sleep(.02)
    cpu.step(instructionwise=True, debug=False)

Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
Trying to execute a non-existant opcode
